In [2]:
import requests
from bs4 import BeautifulSoup
import re
from time import sleep
import random
from __future__ import print_function
import pickle
import pandas as pd
import sqlite3

In [3]:
# Search for all of the possible permutations of data science jobs
# Pull from Indeed.com

search_list = ['data+analyst','business+analyst','data+scientist','data+engineer','data+architect']

job_ids = []

for title in search_list:
    
    print('Parsing {} '.format(title.replace('+',' ')), end='')
    
    test = "https://www.indeed.com.sg/jobs?q={}&l=Singapore&jt=fulltime&start=".format(title)
        
    sleep(random.randint(10, 20))
    
    for page in range(0,110,10): # Change this to make it more managable
        
        #print('Parsing page {}'.format(page))
        
        url = test + '{}'.format(page)
        
        # make request for that page
        sauce = requests.get(url).text
    
        # turn into a BeautifulSoup object
        soup = BeautifulSoup(sauce, 'lxml')
    
        # get the job IDs
        for listing in soup.find_all('a', {'href': re.compile(r'jk\=(\w+)')}):
            job_id = re.findall(r'jk\=(\w+)', listing['href'])[0]
            job_ids.append(job_id)
    
        sleep(random.randint(3, 15))
        
        print('{}, '.format(page), end='')

Parsing data analyst 0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, Parsing business analyst 0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, Parsing data scientist 0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, Parsing data engineer 0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, Parsing data architect 0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 

In [5]:
# Remove all duplicates
job_ids = list(set(job_ids))

In [6]:
len(job_ids)

376

In [ ]:
# Pickle the job ids
# pickle.dump(job_ids, open('job_ids_1604.p','wb'))

In [ ]:
# Unpickle the job ids
# pickle_off = open('job_ids_1604.p','rb')
# job_ids = pickle.load(pickle_off)

In [ ]:
# len(job_ids)

In [14]:
job_ids[:5]

['8c2659cb1181066d',
 'd5073c0120205a65',
 '68122412ba6a2c91',
 '2abc0718a6987824',
 'fe856e889c920d10']

In [15]:
titles = []
company_name = []
job_descriptions = []

#no_data = 'NA'

for id in job_ids:
    
    url = "https://www.indeed.com.sg/viewjob?jk={}".format(id)
    
    sauce = requests.get(url).text
    soup = BeautifulSoup(sauce, 'lxml')
    
    print('--', end='')
    
    sleep(random.randint(2, 5)) # Avoiding the anti-bot mechanism

    for i in soup.find_all('table', {'id': 'job-content'}):
        
        # Extract Job Titles
        try:
            job_title = i.find('b', {'class': 'jobtitle'}).font.text
            titles.append(job_title)
        except Exception:
            pass
        
        # Extract Company Name
        try:
            company = i.find('span', {'class': 'company'}).text
            company_name.append(company)
        except Exception:
            pass
    
        # Extract Job Description
        try:
            job_description = i.find('span', {'class':'summary'}).text.strip()
            job_descriptions.append(job_description)
        except Exception:
            pass
    
        #sleep(random.randint(2, 5)) # Avoiding the anti-bot mechanism
    
    print('>', end='')

-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-

In [16]:
print('Total number of items found: {}'.format(len(titles)))

Total number of items found: 490


In [8]:
print('Total number of items found: {}'.format(len(company_name)))

Total number of items found: 375


In [18]:
print('Total number of items found: {}'.format(len(job_descriptions)))

Total number of items found: 490


In [19]:
titles[-5:]

[u'Architect - Singapore',
 u'ETL Lead',
 u'Senior Data Architect - 6mth RENEWABLE contract - Healthcare IT Leader',
 u'Lead Solution Architect',
 u'Technical Architect']

In [20]:
job_ids[-5:]

['8039003bcb3728af',
 '497f563077c23137',
 'bc2a2b73b04b9b97',
 '8e77b14e4ea3fcf6',
 '89e391ae29af6ce8']

In [21]:
# DataFrame
df = pd.DataFrame({'id': job_ids,'title': titles,'description': job_descriptions})

In [22]:
df.head()

,description,id,title
0,Includes jobs responsible for a variety of act...,8c2659cb1181066d,Pricing Analyst - Asia Pacific Business and Ma...
1,Position Summary:\n\n\nThis position reports t...,d5073c0120205a65,Data Analyst
2,Are you curious? Do you love data? Are you are...,68122412ba6a2c91,Data & Reporting Analyst
3,Dubbed “The Most Innovative Streaming Video Se...,2abc0718a6987824,Data Analyst
4,Do you have a knack for analyzing business pro...,fe856e889c920d10,Business Data Analyst


In [23]:
df['id'].duplicated(keep='first').sum()

84

In [24]:
# df.drop_duplicates(subset='id', keep='first', inplace=True)

In [25]:
# # Check against already mined IDs
mined_ids = pickle.load(open('mined_ids.p','rb'))

In [26]:
# # Check against already scraped data
#
df = df[~df['id'].isin(mined_ids)]

In [27]:
len(df)

77

In [44]:
# Saving to csv instead
df.to_csv('scraped_1604_2.csv', encoding='utf-8', index=False)

In [28]:
# # Saving to a SQL file
connection = sqlite3.connect('job_scraped.db.sqlite')
df.to_sql(name = 'jobs', con = connection, if_exists = 'append', index = False)

In [ ]:
# len(mined_ids)

In [29]:
# Add latest mined job_ids into the mined_job_ids
mined_ids.extend(df['id'].tolist())

In [ ]:
# len(mined_ids)

In [30]:
# Pickle the job ids
pickle.dump(mined_ids, open('mined_ids.p','wb'))

This next part is the second mining for company names for the stopwords.

In [ ]:
company_name = []

for id in job_ids:
    
    url = "https://www.indeed.com.sg/viewjob?jk={}".format(id)
    
    sauce = requests.get(url).text
    soup = BeautifulSoup(sauce, 'lxml')
    
    print('--', end='')
    
    sleep(random.randint(1, 3)) # Avoiding the anti-bot mechanism

    for i in soup.find_all('table', {'id': 'job-content'}):
        
        # Extract Company Name
        try:
            company = i.find('span', {'class': 'company'}).text
            company_name.append(company)
        except Exception:
            pass
    
        #sleep(random.randint(2, 5)) # Avoiding the anti-bot mechanism
    
    print('>', end='')

In [ ]:
# pickle.dump(company_name, open('company_names.p','wb'))